In [282]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [283]:
from batch_sv_system import get_embeds, cosine_sim, compute_plda_score
from utils import key2df, df2dict, compute_eer, get_id2idx
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support

In [347]:
from utils import key2df

embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
si_embeds = np.load(embed_dir+"ln_lda_si_embeds.npy")
sv_embeds = np.load(embed_dir+"ln_lda_sv_embeds.npy")
si_keys = pickle.load(open(embed_dir + "/si_keys.pkl", "rb"))
sv_keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)
si_df = key2df(si_keys)

cohort_ids = np.load("trials/dev940_eval311/dev_cohort_ids.npy")
cohort_embeds = get_embeds(cohort_ids, sv_embeds, sv_id2idx, norm=False)

In [285]:
from ioffe_plda.verifier import Verifier
py_plda_model = Verifier()
py_plda_model = Verifier(pickle.load(open("py_plda_model_ln_lda.pkl", "rb")))

In [338]:
trials = pickle.load(open("trials/dev940_eval311/splited_trials/adapt_enr3_hard_trials.pkl", "rb"))
# trials = pickle.load(open("trials/dev940_eval311/random_enr3xsess_ntar9/small_trials.pkl", "rb"))

In [339]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering

In [351]:
from tqdm import tqdm
import warnings

base_pred_list = []
adapt_pred_list1 = []
adapt_pred_list2 = []
adapt_label_list = []

for t_i in tqdm(range(0, len(trials), 1), total=len(trials)//1):
    trial_info, enr_ids, adapt_trial = trials[t_i]
    adapt_trial = (np.array(adapt_trial.id), np.array(adapt_trial.label))
    init_enr_embeds = get_embeds(enr_ids, sv_embeds, sv_id2idx, norm=False)
    adapt_embeds = get_embeds(adapt_trial[0], sv_embeds, sv_id2idx, norm=False)

    eT = -4.2602
    adapt_scores = py_plda_model.score_avg(init_enr_embeds, adapt_embeds)
    base_adapt_preds = adapt_scores.mean(0) > eT
    base_pred_list.append(base_adapt_preds)

    model_ = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
    
    X = adapt_embeds
    cluster_preds = model_.fit_predict(X)
    mean_plda_scores = dict.fromkeys(range(3))
    for cluster_id in mean_plda_scores.keys():
        mean_plda_scores[cluster_id] =  py_plda_model.multi_sess(
            init_enr_embeds, adapt_embeds[cluster_preds==cluster_id], cov_scaling=True).mean()
    target_cluster = max(mean_plda_scores, key=mean_plda_scores.get)
    adapt_preds = cluster_preds == target_cluster
    adapt_pred_list1.append(adapt_preds)
    
    X = np.concatenate([init_enr_embeds, adapt_embeds])
    cluster_preds = model_.fit_predict(X)
    target_cluster = np.argmax(np.bincount(cluster_preds[:3])) 
    adapt_preds = cluster_preds[3:] == target_cluster
    adapt_pred_list2.append(adapt_preds)
    
    adapt_label_list.append(adapt_trial[1])

100%|██████████| 311/311 [02:10<00:00,  2.38it/s]


In [352]:
prec, recall, _, _ = precision_recall_fscore_support(
    np.concatenate(adapt_label_list), np.concatenate(base_pred_list), average='binary')
print("prec:{:.3f}, recall:{:.3f}".format(prec, recall))

prec:0.359, recall:0.991


In [353]:
prec, recall, _, _ = precision_recall_fscore_support(
    np.concatenate(adapt_label_list), np.concatenate(adapt_pred_list1), average='binary')
print("prec:{:.3f}, recall:{:.3f}".format(prec, recall))

prec:0.343, recall:0.865


In [354]:
prec, recall, _, _ = precision_recall_fscore_support(
    np.concatenate(adapt_label_list), np.concatenate(adapt_pred_list2), average='binary')
print("prec:{:.3f}, recall:{:.3f}".format(prec, recall))

prec:0.332, recall:0.922


In [332]:
for pred, label in zip(base_pred_list, adapt_label_list):
    prec, recall, _, _ = precision_recall_fscore_support(
        label, pred, average='binary')
    print("prec:{:.3f}, recall:{:.3f}".format(prec, recall))

prec:0.193, recall:0.980
prec:0.645, recall:0.980
prec:0.410, recall:0.960
prec:0.247, recall:0.980
prec:1.000, recall:0.780
prec:0.282, recall:0.880
prec:0.397, recall:1.000
prec:0.182, recall:0.400
prec:0.207, recall:0.680
prec:0.671, recall:0.940
prec:1.000, recall:0.900
prec:0.735, recall:1.000
prec:0.209, recall:0.980
prec:0.556, recall:1.000
prec:1.000, recall:1.000
prec:0.266, recall:1.000
prec:1.000, recall:0.880
prec:0.447, recall:0.920
prec:0.258, recall:0.980
prec:0.256, recall:0.820
prec:0.526, recall:0.820
prec:0.979, recall:0.940
prec:1.000, recall:1.000
prec:0.433, recall:0.900
prec:0.360, recall:0.900
prec:0.446, recall:1.000
prec:0.266, recall:1.000
prec:0.602, recall:1.000
prec:0.688, recall:0.880
prec:0.437, recall:0.900
prec:0.778, recall:0.840
prec:0.979, recall:0.940
prec:0.298, recall:0.740
prec:1.000, recall:0.980
prec:0.893, recall:1.000
prec:0.212, recall:0.860
prec:1.000, recall:0.980
prec:0.398, recall:0.900
prec:0.338, recall:1.000
prec:0.742, recall:0.980


In [298]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature

precision, recall, _ = precision_recall_curve(adapt_trial[1], adapt_scores.mean(0))

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

ValueError: Found input variables with inconsistent numbers of samples: [418, 273]